# LightGBM predictions (for predicting metabolites from environmental factors):

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
import optuna
import shap
import os
import matplotlib.pyplot as plt

from matplotlib.colors import LinearSegmentedColormap

def safe_filename(name):
    """
    Convert a string to a safe filename by replacing '/' with '_'.
    """
    return name.replace('/', '_')

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def lightgbm_regression_with_shap(X_df, y_df, n_splits=5, random_state=42, modality_dir='food'):
    """
    Perform LightGBM regression with cross-validation and SHAP analysis.
   
    Parameters:
    X_df (DataFrame): Feature dataframe
    y_df (DataFrame): Target dataframe with a single column
    n_splits (int): Number of splits for KFold cross-validation
    random_state (int): Random state for reproducibility
   
    Returns:
    dict: A dictionary containing the final model, cross-validation scores, SHAP values, and feature importance
    """
   
    ensure_dir(f'./figs/{modality_dir}')
    # Create a directory for this target variable
    # Get the target variable name
    if isinstance(y_df, pd.DataFrame):
        target_name = y_df.columns[0]
    elif isinstance(y_df, pd.Series):
        target_name = y_df.name if y_df.name is not None else 'target'
    else:
        target_name = 'target'
    output_dir = f'./figs/{modality_dir}/{safe_filename(target_name)}'
    ensure_dir(output_dir)
   
    # Convert dataframes to numpy arrays
    X = X_df.values
    y = y_df.values.ravel()
   
    # Initialize the LightGBM model
    model = LGBMRegressor(random_state=random_state)
   
    # Perform cross-validation
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    y_true_all = []
    y_pred_all = []
    fold_r2_scores = []
   
    for fold, (train_idx, test_idx) in enumerate(cv.split(X), 1):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
       
        # Train the model
        model.fit(X_train, y_train)
       
        # Make predictions on the test set
        y_pred = model.predict(X_test)
       
        # Append the true labels and predictions to the overall lists
        y_true_all.extend(y_test)
        y_pred_all.extend(y_pred)
       
        # Calculate the R^2 score for the current fold
        fold_r2 = r2_score(y_test, y_pred)
        fold_r2_scores.append(fold_r2)
        print(f"Fold {fold} R^2: {fold_r2:.4f}")
   
    # Calculate the overall R^2 score
    overall_r2 = r2_score(y_true_all, y_pred_all)
    print(f"Overall R^2: {overall_r2:.4f}")
   
    # Train the final model on the entire dataset
    final_model = LGBMRegressor(random_state=random_state)
    final_model.fit(X, y)
   
    # SHAP analysis
    explainer = shap.TreeExplainer(final_model)
    shap_values = explainer.shap_values(X)
   
    # Plot SHAP beeswarm summary plot
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X, plot_type="dot", feature_names=X_df.columns, show=False)
    plt.tight_layout()
    plt.show()
    # plt.close()
   
    print("Cross-validation and SHAP analysis completed.")
    print("SHAP beeswarm summary plot has been saved in the 'figs' folder.")
   
    # Calculate mean absolute SHAP values for each feature
    mean_shap_values = np.abs(shap_values).mean(axis=0)
    feature_importance = pd.DataFrame({'feature': X_df.columns, 'importance': mean_shap_values})
    feature_importance = feature_importance.sort_values('importance', ascending=False).reset_index(drop=True)
   
    # # Print top 10 contributing features
    # print("\nTop 5 contributing features according to SHAP analysis:")
    # for i, row in feature_importance.head(5).iterrows():
    #     print(f"{i+1}. {row['feature']}: {row['importance']:.4f}")
    #
    return {
        'final_model': final_model,
        'fold_r2_scores': fold_r2_scores,
        'overall_r2': overall_r2,
        'shap_values': shap_values,
        'feature_importance': feature_importance,
        'X_df': X_df,
        'y_df': y_df,
        'y_pred_all': y_pred_all,
        'y_true_all': y_true_all,
    }


# Figure 4 A Plot beeswarm colorful plot of prediction performance

In [ ]:

def plot_coefficient_performance(df1, df2, df3, df4, df5, column_name='Coefficient_of_determination', x_ticks=None):
    """
    Plot grey box plots with colored swarm plots for Coefficient of Determination from 4 DataFrames.
    Takes top 50 values less than 1 from each DataFrame, excluding 'age', 'gender', and 'bmi' rows.
   
    Parameters:
    df1, df2, df3, df4 (pd.DataFrame): Input DataFrames containing the data to plot
    column_name (str): Name of the column containing the Coefficient of Determination values
    x_ticks (list): Custom x-axis tick labels. If None, default labels will be used.
   
    Returns:
    None (displays the plot)
    """
    # Combine the DataFrames
    dfs = [df1, df2, df3, df4, df5]
    data = []
    excluded_indexes = ['age', 'gender', 'bmi']
   
    for i, df in enumerate(dfs, 1):
        # Filter out excluded indexes, values less than 1, and take top 50
        filtered_df = df[~df.index.isin(excluded_indexes)]
        valid_values = filtered_df[filtered_df[column_name] < 1][column_name].nlargest(50)
        data.extend([(f'Table {i}', val) for val in valid_values])

    # Create a new DataFrame from the combined data
    plot_df = pd.DataFrame(data, columns=['Table', column_name])

    # Set up the plot
    plt.figure(figsize=(6, 4))
    sns.set_style("whitegrid")

    # Create grey box plot
    sns.boxplot(x='Table', y=column_name, data=plot_df,
                width=0.5, fliersize=0, color='white', linewidth=1.5,
                boxprops=dict(edgecolor="grey"))

    # Create colored swarm plot
    palette = sns.color_palette("hls", len(dfs))  # You can change this to any other color palette
    sns.swarmplot(x='Table', y=column_name, data=plot_df,
                  size=3, palette=palette)

    # Customize the plot
    # plt.title('Explained Variance (R²)', fontsize=16)
    plt.xlabel('', fontsize=12)
    plt.ylabel('Explained Variance (R²)', fontsize=12)
    plt.ylim(0, 0.8)  # Set y-axis limit from 0 to 0.8
    plt.yticks(np.arange(0, 0.9, 0.2))

    # Set custom x-ticks if provided
    if x_ticks:
        plt.xticks(range(len(x_ticks)), x_ticks)
    else:
        plt.xticks(range(4), [f'({plot_df[plot_df["Table"] == f"Table {i+1}"].shape[0]})' for i in range(4)])
       
    # set left and bottom spines to black
    plt.gca().spines['left'].set_color('black')
    plt.gca().spines['bottom'].set_color('black')

    # Show the number of points below x-axis labels
    ax = plt.gca()
    ax.tick_params(axis='x', labelsize=10)
   
    plt.tight_layout()
    plt.show()


# Figure 4 c, e Plot Scatter plot of predicted vs. True colored by top SHAPs

In [ ]:
from scipy import stats
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import pandas as pd

def plot_predicted_vs_true(results, num_features=5, modality_dir='food', tick_font_size=6, dot_size=10, target_nick=None):
    """
    Create scatter plots of predicted vs true values, colored by the top SHAP features.
    Use a white-to-red colormap for feature values with black outer rim for each dot.
    Save plots in a directory named after the target variable.
    Also save a text file with Pearson correlations and p-values for top features.
   
    Parameters:
    results (dict): Output from lightgbm_regression_with_shap function
    num_features (int): Number of top features to plot
    modality_dir (str): Directory name for the modality
    tick_font_size (int): Font size for axis tick labels
    dot_size (float): Size of the dots in the scatter plot
    target_nick (str): Nickname for the target variable (optional)
   
    Returns:
    None
    """
    font_size = 7
    feature_importance = results['feature_importance']
   
    # Get predictions for the entire dataset
    y_pred = results['y_pred_all']
    y_true = results['y_true_all']
    y_df = results['y_df']
    X_df = results['X_df']
   
    # Get the target variable name
    if isinstance(y_df, pd.DataFrame):
        target_name = y_df.columns[0]
    elif isinstance(y_df, pd.Series):
        target_name = y_df.name if y_df.name is not None else 'target'
    else:
        target_name = 'target'
   
    # Create a directory for this target variable
    output_dir = f'./figs/{modality_dir}/{safe_filename(target_name)}'
    ensure_dir(output_dir)
   
    # Get the top features
    top_features = feature_importance.head(num_features)['feature'].tolist()
   
    # Create a custom colormap from white to red
    colors = ['#FFFFFF', '#FFA07A', '#FF0000']  # White to Light Salmon to Red
    n_bins = 100  # Number of color gradations
    cmap = LinearSegmentedColormap.from_list("custom_red", colors, N=n_bins)
   
    # Prepare a list to store correlation results
    correlation_results = []
   
    for feature in top_features:
        feature_values = X_df[feature]
       
        if modality_dir == 'microbiome':
            short_name = feature.split('|')[-2]
        else:
            short_name = feature
       
        plt.figure(figsize=(2.5, 2.5), facecolor='white')
        ax = plt.gca()
        ax.set_facecolor('white')
       
        # Set spine visibility and color
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_color('black')
        ax.spines['left'].set_color('black')
        ax.spines['bottom'].set_linewidth(0.5)
        ax.spines['left'].set_linewidth(0.5)
       
        # Configure ticks
        ax.tick_params(direction='out', length=2, width=0.5, colors='black')
        ax.tick_params(axis='x', which='both', bottom=True)
        ax.tick_params(axis='y', which='both', left=True)
        ax.grid(False)
       
        scatter = plt.scatter(y_true, y_pred, c=feature_values, cmap=cmap,
                              alpha=0.6, edgecolors='black', linewidth=0.2,
                              s=dot_size)
       
        # Add perfect prediction line
        min_val = min(min(y_true), min(y_pred))
        max_val = max(max(y_true), max(y_pred))
        plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=0.5)
       
        if target_nick is None:
            target_nick = target_name
       
        plt.xlabel(f'Measured', fontsize=font_size)
        plt.ylabel(f'Predicted', fontsize=font_size)
        # plt.title(f'{target_nick}  $C_9H_8O_6S$', fontsize=font_size)
        plt.title(f'{target_nick}', fontsize=font_size)
        cbar = plt.colorbar(scatter)
        cbar.set_label(f'{short_name} [mg/day]', fontsize=font_size-1)
       
        scatter.set_clim(feature_values.min(), feature_values.max())
       
        plt.tick_params(axis='both', which='major', labelsize=tick_font_size)
        cbar.ax.tick_params(labelsize=tick_font_size)
       
        plt.tight_layout()
        plt.close()
       
        # Calculate Pearson correlation and p-value
        correlation, p_value = stats.pearsonr(feature_values, y_pred)
        correlation_results.append({
            'Feature': short_name,
            'Correlation': correlation,
            'P-value': p_value
        })
   
    # Save correlation results to a text file
    correlation_df = pd.DataFrame(correlation_results)
    correlation_df = correlation_df.sort_values('Correlation', ascending=False)
    correlation_file = f'{output_dir}/feature_correlations.txt'
    with open(correlation_file, 'w') as f:
        f.write(f"Pearson correlations and p-values for top {num_features} features with predicted {target_nick}:\n\n")
        f.write(correlation_df.to_string(index=False))
   
    print(f"Scatter plots of predicted vs true values for top {num_features} features have been saved in '{output_dir}'.")
    print(f"Correlation results have been saved to '{correlation_file}'.")

# Histogram plot Figure 4 b

In [ ]:


def plot_coefficient_histogram(df, main_fontsize=7, axis_fontsize=7):
   filtered_df = df[(df['Coefficient_of_determination'] != 1) &
                    (~df['Size'].isin(['bmi', 'age']))]
   
   plt.rcParams.update({'font.size': main_fontsize})
   fig, ax = plt.subplots(figsize=(3.5, 3.5))
   
   # Remove grid
   ax.grid(False)
   
   n, bins, patches = ax.hist(filtered_df['Coefficient_of_determination'],
                              bins=20,
                              edgecolor='black',
                              color='salmon',
                              rwidth=0.6)
   
   ax.set_xlabel('EV of full model', fontsize=axis_fontsize)
   ax.set_ylabel('Number of metabolites', fontsize=axis_fontsize)
   ax.set_title('Distribution of Coefficient of Determination', fontsize=axis_fontsize)
   ax.set_xlim(0, 0.7)
   
   inset_ax = ax.inset_axes([0.5, 0.5, 0.35, 0.35])
   inset_ax.grid(False)
   inset_ax.hist(filtered_df['Coefficient_of_determination'],
                 bins=20,
                 edgecolor='black',
                 color='salmon',
                 rwidth=0.5)
   
   inset_ax.set_xlim(0.3, 0.6)
   inset_ax.set_xticks(np.arange(0.3, 0.61, 0.1))
   inset_ax.set_xticklabels([f'{x:.1f}' for x in np.arange(0.3, 0.61, 0.1)])
   inset_ax.set_ylim(0, 100)
   
   plt.tight_layout()
   plt.show()

# PDAC and HEALTHY METABOLOMICS

In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from scipy.stats import mannwhitneyu
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi
import numpy as np

In [ ]:
ten_k_df = pd.read_csv("10k_cohort")  #### Basic partici[ants data, like age, gender and BMI
metabolomics_df = pd.read_csv("normalized_metabolomics_data")  #### LCMS Metabolomics

# Process healthy controls 

In [ ]:
metabolomics_lvs = pd.DataFrame(metabolomics_df.T.index, columns=['name_dna'])
healthy_controls = pd.merge(ten_k_df, metabolomics_lvs, on='name_dna', how='inner')
healthy_controls = healthy_controls[healthy_controls['research_stage'] == 'baseline']
healthy_controls.dropna(subset=['RegistrationCode', 'research_stage', 'age', 'gender', 'bmi'], inplace=True)
healthy_controls.drop(['primary_tube_id_dna', 'participant_id', 'yob', 'research_stage'], axis=1, inplace=True)
healthy_controls = healthy_controls.rename(columns={'name_dna': 'lv_code'})
healthy_controls = healthy_controls.astype({'age': 'int64'}, copy=False)

# Process PDAC Patients

In [ ]:
pdac_meta = pd.read_excel("PDAC_patient_metadata", sheet_name='user_metadata')  
pdac_meta.columns = pdac_meta.columns.str.strip()
columns_to_keep = ['SPC_ID', 'Gender', 'Age', 'BMI'] 
pdac_meta = pdac_meta[columns_to_keep]

In [ ]:
# Integrating sample plate metadata for PDAC patients.

sheet_names = ['Pancreatic_Plate1', 'Pancreatic_Plate2', 'Pancreatic_Plate3', 'Pancreatic_Plate4']
barcode_columns = ['LV Barcodes Plate1_1', 'LV Barcodes Plate2_1', 'LV Barcodes Plate3_1', 'LV Barcodes Plate4_1']

plates = []
for sheet, barcode_col in zip(sheet_names, barcode_columns):
    plate = pd.read_excel("PDAC_sample_plate_metadata", sheet_name=sheet,  
    sheet_name=sheet
    )
    plate.columns = plate.columns.str.replace('\n', '')  
    plate = plate[['SPC_ID', barcode_col, 'Date of sample aqcuisition']].rename(columns={barcode_col: 'lv_code'})
    plates.append(plate)

plates_df = pd.concat(plates, axis=0, ignore_index=True)

pdac_samples = pd.merge(pdac_meta, plates_df, on='SPC_ID', how='left')

pdac_samples.dropna(inplace=True)
pdac_samples.sort_values(by=['SPC_ID', 'Date of sample aqcuisition'], inplace=True)
pdac_samples.drop_duplicates(subset='SPC_ID', keep='first', inplace=True)
pdac_samples.rename(columns={'Gender': 'gender', 'Age': 'age', 'BMI': 'bmi'}, inplace=True)
pdac_samples['gender'].replace({'F': 0.0, 'M': 1.0}, inplace=True)

# Matching helthy controls to pdac patients

In [ ]:
healthy_controls['PDAC sample'] = 0  
pdac_samples['PDAC sample'] = 1  

# Features for matching (covariates: age, gender, bmi)
features = ['age', 'gender', 'bmi']

# Standardizing the features
scaler = StandardScaler()
pdac_features_scaled = scaler.fit_transform(pdac_samples[features])
healthy_features_scaled = scaler.transform(healthy_controls[features])

# Matching 1:50
nbrs = NearestNeighbors(n_neighbors=50, algorithm='ball_tree').fit(healthy_features_scaled)

distances, indices = nbrs.kneighbors(pdac_features_scaled)

matched_results = []
for patient_idx, match_indices in enumerate(indices):
    pdac_patient = pdac_samples.iloc[patient_idx]
    
    matched_patients = healthy_controls.iloc[match_indices].copy()
    
    for _, healthy_control in matched_patients.iterrows():
        matched_results.append({
            'PDAC ID': pdac_patient['SPC_ID'],  
            'PDAC Age': pdac_patient['age'],  
            'PDAC Gender': pdac_patient['gender'],  
            'PDAC BMI': pdac_patient['bmi'],  
            'PDAC LV Code': pdac_patient['lv_code'],  
            'Matched Healthy Control': healthy_control['RegistrationCode'],  
            'Healthy Control LV Code': healthy_control['lv_code'],  
            'Healthy Age': healthy_control['age'],  
            'Healthy Gender': healthy_control['gender'], 
            'Healthy BMI': healthy_control['bmi'],  
        })

matched_df = pd.DataFrame(matched_results)

# The individual-level metabolic signature of PDAC

In [ ]:
matching_results = matched_df.copy()
matching_results.drop(['PDAC Age', 'PDAC Gender', 'PDAC BMI', 'Healthy Age', 'Healthy Gender', 'Healthy BMI'], axis=1, inplace=True)

healthy_control_lv_codes = matched_df['Healthy Control LV Code']
healthy_metabolites = metabolomics_df[['Compound'] + healthy_control_lv_codes.tolist()]

pdac_lv_codes = matched_df['PDAC LV Code']
pdac_metabolites = metabolomics_df[['Compound'] + pdac_lv_codes.tolist()]

In [ ]:
# calculate the mean and std of metabolite values for each PDAC patient based on their matched healthy controls.

control_means = {}
control_stds = {}

for pdac_id in matched_df['PDAC ID']:  
    
    healthy_lvs = matched_df[matched_df['PDAC ID'] == pdac_id]['Healthy Control LV Code']
    healthy_data = metabolomics_df[['Compound'] + healthy_lvs.tolist()]
    if pdac_id == 'SPC_481':
        print(healthy_data)
    
    control_means[pdac_id] = healthy_data.iloc[:, 1:].mean(axis=1)
    control_stds[pdac_id] = healthy_data.iloc[:, 1:].std(axis=1)

In [ ]:
# Calculate z-scores for PDAC patients

z_scores = {}

for idx, row in matched_df.iterrows():
    pdac_id = row['PDAC ID']
    pdac_lv_code = row['PDAC LV Code']
    
    pdac_data = metabolomics_df[['Compound', pdac_lv_code]]
    
    # Calculating z-scores for each metabolite as the deviation from the healthy control mean, standardized by the standard deviation.
    z_scores[pdac_id] = (pdac_data.iloc[:, 1] - control_means[pdac_id]) / control_stds[pdac_id]

z_scores_df = pd.DataFrame(z_scores)

z_scores_df.insert(0, 'Compound', pdac_data['Compound'])
z_scores_df

# MannWhitney U test for significant metabolites

In [ ]:
healthy_metabolites_full_lvs = healthy_controls['lv_code'].tolist()
healthy_metabolites_full = metabolomics_df[['Compound'] + healthy_metabolites_full_lvs]

pdac_metabolites_full_lvs = plates_df.dropna(subset=['lv_code'])['lv_code']
pdac_metabolites_full_lvs = [lv for lv in pdac_metabolites_full_lvs if lv != 'EMPTY']
pdac_metabolites_full = metabolomics_df[['Compound'] + pdac_metabolites_full_lvs]

In [ ]:
results = []
pdac_depleted = True

for metabolite in pdac_metabolites_full['Compound']:
    
    pdac_values = pdac_metabolites_full[pdac_metabolites_full['Compound'] == metabolite].iloc[:, 1:].values.flatten()
    healthy_values = healthy_metabolites_full[healthy_metabolites_full['Compound'] == metabolite].iloc[:, 1:].values.flatten()
    
    # Performing Mann-Whitney U test to compare PDAC and healthy groups.
    u_statistic, p_value = mannwhitneyu(pdac_values, healthy_values, alternative='two-sided')
    
    if pdac_values.mean() - healthy_values.mean() < 0: # The metabolite is PDAC-depleted
        pdac_depleted = True
    else:
        pdac_depleted = False

    results.append({
        'Compound': metabolite,
        'U-statistic': u_statistic,
        'P-value': p_value,
        'PDAC_Mean': pdac_values.mean(),
        'Healthy_Mean': healthy_values.mean(),
        'Mean_Difference': pdac_values.mean() - healthy_values.mean(),
        'PDAC Depleted': pdac_depleted
    })
results

metabolites_directions = pd.DataFrame(results)
metabolites_directions

In [ ]:
# Filter significantly deviating metabolites
metabolites_directions = metabolites_directions[metabolites_directions['P-value'] < 0.05]

merged_z_scores_df = pd.merge(z_scores_df, metabolites_directions, on='Compound', how='inner')
merged_z_scores_df = merged_z_scores_df.drop(['U-statistic', 'P-value', 'PDAC_Mean', 'Healthy_Mean', 'Mean_Difference'], axis=1)

In [ ]:
# Focus only on relevant directional changes (otherwise replace the value with 0).
def modify_row(row):
    numeric_cols = row.index.difference(['Compound', 'PDAC Depleted']) 

    # Set positive values to 0 for PDAC-depleted metabolites, and negative values to 0 otherwise.
    if row['PDAC Depleted']:  
        row[numeric_cols] = row[numeric_cols].apply(lambda x: 0 if x > 0 else x)
    else:  
        row[numeric_cols] = row[numeric_cols].apply(lambda x: 0 if x < 0 else x)
    return row

merged_z_scores_df = merged_z_scores_df.apply(modify_row, axis=1)
merged_z_scores_df

In [ ]:
pdac_enriched = merged_z_scores_df[merged_z_scores_df['PDAC Depleted'] == False]
pdac_depleted = merged_z_scores_df[merged_z_scores_df['PDAC Depleted'] == True]

In [ ]:
# Extract the top 100 PDAC-depleted metabolites for each PDAC patient, based on the z-scores.
top_depleted_metabolites_sorted = pd.DataFrame()

for col in pdac_depleted.columns[1:-1]:  
    top_100_idx = pdac_depleted[col].abs().nlargest(100).index
    top_100_values = pdac_depleted.loc[top_100_idx, col]
    
    top_100_metabolites = pdac_depleted.loc[top_100_idx, 'Compound'].str.replace('.', '_').str.replace('/', '_')
    
    sorted_top_100 = pd.DataFrame({f'{col}': top_100_metabolites.values, f'{col}_zscore': top_100_values.values})
    
    top_depleted_metabolites_sorted = pd.concat([top_depleted_metabolites_sorted, sorted_top_100], axis=1)

top_depleted_metabolites_sorted

In [ ]:
# Extract the top 100 PDAC-enriched metabolites for each PDAC patient, based on the z-scores.

top_enriched_metabolites_sorted = pd.DataFrame()

for col in pdac_enriched.columns[1:-1]: 
    top_100_idx = pdac_enriched[col].nlargest(100).index

    top_100_values = pdac_enriched.loc[top_100_idx, col]

    top_100_metabolites = pdac_enriched.loc[top_100_idx, 'Compound'].str.replace('.', '_').str.replace('/', '_')

    sorted_top_100 = pd.DataFrame({f'{col}': top_100_metabolites.values, f'{col}_zscore': top_100_values.values})

    top_enriched_metabolites_sorted = pd.concat([top_enriched_metabolites_sorted, sorted_top_100], axis=1)

top_enriched_metabolites_sorted

# Involve predicted metabolic determinants

In [ ]:
microbiome = pd.read_csv("microbiome_prediction_results")
diet = pd.read_csv("diet_prediction_results")
lifestyle = pd.read_csv("lifestyle_prediction_results")
clinical = pd.read_csv("clinical_prediction_results")

microbiome.index.name = 'microbiome'
diet.index.name = 'diet'
lifestyle.index.name = 'lifestyle'
clinical.index.name = 'clinical'

In [ ]:
# Add a binary indicator column to each feature DataFrame 
# (to indicate if the explained variance score for a given metabolite exceeds a significance threshold of 0.05).

def add_binary_EV_column(feature_df):
    feature_df.rename(columns={'Unnamed: 0': 'Compound'}, inplace=True)
    feature_df[f'is_EV_{feature_df.index.name}'] = 0
    feature_df.loc[feature_df['explained_variance_score'] > 0.05, f'is_EV_{feature_df.index.name}'] = 1
    
    columns_to_keep = ['Compound', f'is_EV_{feature_df.index.name}']
    feature_df.drop(columns=[col for col in feature_df.columns if col not in columns_to_keep], inplace=True)

features_dfs = [microbiome, diet, lifestyle, clinical]

for feature_df in features_dfs:
    add_binary_EV_column(feature_df)

In [ ]:
# Creating individual DataFrames for each PDAC pateint, with metabolites and their corresponding z-scores,

def create_individual_spc_dfs(source_df):
    individual_spc_dfs = []

    spc_ids = [col for col in source_df.columns if '_zscore' not in col]

    for spc_id in spc_ids:

        df = source_df[[spc_id, f'{spc_id}_zscore']].copy()

        df.columns = ['Compound', f'{spc_id}_zscore']
        
        individual_spc_dfs.append(df)

    return individual_spc_dfs

enriched_individual_spc_dfs = create_individual_spc_dfs(top_enriched_metabolites_sorted)
depleted_individual_spc_dfs = create_individual_spc_dfs(top_depleted_metabolites_sorted)

In [ ]:
# Merge each determinant with each patient's metabolite DataFrame for integrated analysis.

def merge_features_to_spc_dfs(individual_spc_dfs, features_dfs):

    for i, spc_df in enumerate(individual_spc_dfs):
        spc_df_copy = spc_df.copy()  
        
        for feature_df in features_dfs:
            spc_df_copy = pd.merge(spc_df_copy, feature_df, on='Compound', how='inner')  
        
        individual_spc_dfs[i] = spc_df_copy
    
    return individual_spc_dfs

enriched_individual_spc_dfs = merge_features_to_spc_dfs(enriched_individual_spc_dfs, features_dfs)
depleted_individual_spc_dfs = merge_features_to_spc_dfs(depleted_individual_spc_dfs, features_dfs)

In [ ]:
# Compute average determinants scores for each patient by weighting z-scores with binary determinants indicators.

def build_feature_scores(individual_spc_dfs):
    feature_scores = {}

    for df in individual_spc_dfs:
        for idx, row in df.iterrows():
            spc_id = df.columns[1].replace('_zscore', '')

            if spc_id not in feature_scores:
                feature_scores[spc_id] = {
                    'microbiome_score': [], 
                    'diet_score': [], 
                    'lifestyle_score': [], 
                    'clinical_score': []
                }

            # Calculating the weighted z-score for each determinant.
            zscore = abs(row[f'{spc_id}_zscore']) 
            
            feature_scores[spc_id]['microbiome_score'].append(row['is_EV_microbiome'] * zscore)
            feature_scores[spc_id]['diet_score'].append(row['is_EV_diet'] * zscore)
            feature_scores[spc_id]['lifestyle_score'].append(row['is_EV_lifestyle'] * zscore)
            feature_scores[spc_id]['clinical_score'].append(row['is_EV_clinical'] * zscore)

        # Computing the mean score for each determinant.
        feature_scores[spc_id]['microbiome_score'] = statistics.mean(feature_scores[spc_id]['microbiome_score'])
        feature_scores[spc_id]['diet_score'] = statistics.mean(feature_scores[spc_id]['diet_score'])
        feature_scores[spc_id]['lifestyle_score'] = statistics.mean(feature_scores[spc_id]['lifestyle_score'])
        feature_scores[spc_id]['clinical_score'] = statistics.mean(feature_scores[spc_id]['clinical_score'])

    return pd.DataFrame(feature_scores).T

enriched_feature_scores = build_feature_scores(enriched_individual_spc_dfs)
depleted_feature_scores = build_feature_scores(depleted_individual_spc_dfs)

# Plots

Determinants score distribution plot (Fig. 4d)

In [ ]:
# Visualize kde plots of determinant scores for PDAC-enriched and PDAC-depleted metabolites.

labels_fontsize = 7
tick_fontsize = 7
title_padding = 3
width_in = 130 / 25.4
height_in = 60 / 25.4
colors = ['green', 'red', 'purple', 'blue']
scores = ['microbiome_score', 'diet_score', 'lifestyle_score', 'clinical_score']

fig, axes = plt.subplots(1, 2, figsize=(width_in, height_in), sharey=True)

for score, color in zip(scores, colors):
    sns.kdeplot(enriched_feature_scores[score], label=score.split('_')[0].title(), ax=axes[0], color=color, lw=2)

axes[0].set_title('PDAC-enriched metabolites', fontsize=labels_fontsize, pad=title_padding+2)
axes[0].set_xlabel('Mean determinant score', fontsize=labels_fontsize, labelpad=title_padding)
axes[0].set_ylabel('Participant density', fontsize=labels_fontsize, labelpad=title_padding)
axes[0].tick_params(labelsize=tick_fontsize)

for score, color in zip(scores, colors):
    sns.kdeplot(depleted_feature_scores[score], label=score.split('_')[0].title(), ax=axes[1], color=color, lw=2)

axes[1].set_title('PDAC-depleted metabolites', fontsize=labels_fontsize, pad=title_padding+2)
axes[1].set_xlabel('Mean determinant score', fontsize=labels_fontsize, labelpad=title_padding)
axes[1].tick_params(labelsize=tick_fontsize)

axes[1].legend(fontsize=tick_fontsize, loc='upper left', bbox_to_anchor=(0.511, 1))

plt.tight_layout()
plt.savefig("combined_pdac_metabolites.png", dpi=300, bbox_inches='tight')
plt.show()

# Standrdizing the scores based on the features in each patient

In [ ]:
standardized_depleted = depleted_feature_scores.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
standardized_enriched = enriched_feature_scores.apply(lambda x: (x - x.mean()) / x.std(), axis=0)

standardized_depleted.columns = [f"{col}_Depleted" for col in standardized_depleted.columns]
standardized_enriched.columns = [f"{col}_Enriched" for col in standardized_enriched.columns]

merged_standardized_df = pd.concat([standardized_depleted, standardized_enriched], axis=1)

# Heatmap (Fig. 4f)

In [ ]:
# Visualizing standardized determinant scores for PDAC-enriched and PDAC-depleted metabolites.

merged_standardized_df.columns = [
    'Microbiome', 'Diet', 'Lifestyle', 'Clinical',
    'Microbiome', 'Diet', 'Lifestyle', 'Clinical'
]
plt.figure(figsize=(3.5, 3.5))

ax = sns.heatmap(merged_standardized_df, cmap='RdBu_r', linewidths=0.5, linecolor='white',
                 cbar_kws={'label': 'Standardized score', 'ticks': [-2, -1, 0, 1, 2]},
                 yticklabels=False)

ax.set_xticklabels(ax.get_xticklabels(), fontsize=7, rotation=45, ha='center')

plt.xlabel('Metabolic determinants', fontsize=7, labelpad=12)
plt.ylabel('PDAC Patients', fontsize=7, labelpad=12)

plt.xticks(rotation=45, ha='right')

plt.text(2, -0.5, 'PDAC-depleted', fontsize=7, ha='center')
plt.text(6.1, -0.5, 'PDAC-enriched', fontsize=7, ha='center')

cbar = ax.collections[0].colorbar
cbar.ax.yaxis.label.set_rotation(270)
cbar.ax.yaxis.labelpad = 14
cbar.set_label('Standardized score', fontsize=7)
cbar.ax.tick_params(labelsize=7)

plt.axvline(x=4, color='black', linewidth=1, linestyle='--')

plt.tight_layout()
plt.savefig("heat_map.png", dpi=300, bbox_inches='tight')
plt.show()

# Radar plot (Fig. 4g)

In [ ]:
# Comparing metabolic determinant scores for two PDAC patients, 
# showing enriched and depleted categories of all 4 determinants.

# We choose 2 patients with similar covariates.
patients = ['SPC_425', 'SPC_667']  

merged_standardized_df.columns = [
    'Microbiome\n(PDAC-depleted)', 'Diet\n(PDAC-depleted)', 'Lifestyle\n(PDAC-depleted)', 'Clinical Data\n(PDAC-depleted)',
    'Microbiome\n(PDAC-enriched)', 'Diet\n(PDAC-enriched)', 'Lifestyle\n(PDAC-enriched)', 'Clinical Data\n(PDAC-enriched)'
]

categories = [
    'Microbiome\n(PDAC-depleted)', 'Diet\n(PDAC-depleted)', 'Lifestyle\n(PDAC-depleted)', 'Clinical Data\n(PDAC-depleted)',
    'Microbiome\n(PDAC-enriched)', 'Diet\n(PDAC-enriched)', 'Lifestyle\n(PDAC-enriched)', 'Clinical Data\n(PDAC-enriched)'
]
N = len(categories)

angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1] 

# Standardized determinant scores for the 2 selected patients.
patient1_values = merged_standardized_df.loc[patients[0], categories].tolist()
patient1_values += patient1_values[:1]

patient2_values = merged_standardized_df.loc[patients[1], categories].tolist()
patient2_values += patient2_values[:1]

fig, ax = plt.subplots(figsize=(3.5, 3.5), subplot_kw=dict(polar=True))

ax.plot(angles, patient1_values, linewidth=0.7, linestyle='solid', label='Participant 1', color='#1f77b4')
ax.fill(angles, patient1_values, color='#1f77b4', alpha=0.2)

ax.plot(angles, patient2_values, linewidth=0.7, linestyle='solid', label='Participant 2', color='#ff7f0e')
ax.fill(angles, patient2_values, color='#ff7f0e', alpha=0.2)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=7, ha='center')

tick_values = [-1, 0, 1, 2]  
ax.set_yticks(tick_values)  
ax.set_yticklabels([str(int(tick)) for tick in tick_values], fontsize=7, color='gray') 

ax.set_ylim(-2, 2) 

ax.yaxis.grid(True, linestyle='--', color='gray', linewidth=0.7)

for label, angle in zip(ax.get_xticklabels(), angles[:-1]):
    angle_deg = np.degrees(angle)
    if angle_deg == 90 or angle_deg == 270:
        label.set_horizontalalignment('center')
        label.set_position((0, -0.01))  
    if angle_deg < 90 or angle_deg > 270:
        label.set_horizontalalignment('left')
        label.set_position((0.01, 0.1))  
    if 90 < angle_deg < 270:
        label.set_horizontalalignment('right')
        label.set_position((-0.01, 0.1)) 

plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.5), fontsize=7)
plt.tight_layout()
plt.savefig("radar_plot.png", dpi=300, bbox_inches='tight')
plt.show()